In [ ]:
!pip install nltk==3.4 ## needs latest version
!pip install pyodbc
!apt-get install virtuoso-opensource
!pip install SPARQLWrapper
!pip install sparql_dataframe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436399 sha256=bc26b1bdaa016d4890942a3cbff507c842de52d73873b28f8a422d08e505ecf2
  Stored in directory: /root/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 280 kB 5.2 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287319 sha256=b0691d361e668e6cd7e1c34bbcfd64a786fd933827e9cc47b9f4b36190a1b43e
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef5978787

In [ ]:
import requests
import os 
import re
import logging
import sys
import pyodbc
import hashlib
import pandas as pd
import numpy as np
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST, GET
from SPARQLWrapper import JSON, INSERT, DELETE
import sparql_dataframe
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def Remove(duplicate):
    final = []
    for num in duplicate:
        if num not in final:
            final.append(num)
    return final

In [ ]:
def wordNet(word):
  keyword = word.split(" ")

  listWN = []
  for key in keyword:
    listKeyword = []
    synonyms = wn.synsets(key)
    for s in synonyms:
      helper = s.name().replace(".n.", "").replace(".v.", "").replace(".r.", "").replace(".a.", "")
      result = ''.join([i for i in helper if not i.isdigit()])
      if result not in keyword:
        syn1 = wn.synsets(key)[0]
        syn2 = wn.synsets(result)[0]
        score = syn1.wup_similarity(syn2)
        listKeyword.append((result, score))
    listWN.append((key, listKeyword))

  finalListWN =[]
  for l in listWN:
    finalListWN.append((l[0], Remove(l[1])))


  return finalListWN

In [ ]:
class findSimilarity():

    def __init__(self, entities, properties, perceived, weights):
        self.entities = entities
        self.properties = properties
        self.perceived = perceived
        self.weights = weights
        
    def Remove(self, duplicate):
        final = []
        for num in duplicate:
            if num not in final:
                final.append(num)
        return final

    def hasNumbers(self, inputString):
        return any(char.isdigit() for char in inputString)

    def cleaning_entities(self):
        list1 = []
        entity_property, second_cleaning = {}, []
        for i in range(len(self.entities)):
            first_cleaning = self.entities[i].replace('/a/[', '').replace(']', '')
            for j in range(len(self.properties)):
                if self.properties[j] + '/' in first_cleaning:
                    second_cleaning = first_cleaning.replace(self.properties[j] + '/', '').replace(',', '')
                    if entity_property.get(self.properties[j]) == None:
                        entity_property.setdefault(self.properties[j], []).append(second_cleaning)
                    else:
                        entity_property[self.properties[j]].append(second_cleaning)
                    break
            list1.append((second_cleaning, self.weights[i]))


        wanted_properties = ["/r/RelatedTo"]

        list_key_help = []
        for key in entity_property.keys():
            if key in wanted_properties:
                list_key_help.append(key)

        entity_property_keep = {}
        for property in wanted_properties:
            if property in list_key_help:
                entity_property_keep[property] = entity_property[property]
            else:
                entity_property_keep[property] = []


        return list1, entity_property_keep

    def cleaning_entities2(self, entities_observed, perceived_weights):

        for observed in entities_observed:
            for entity in range(len(entities_observed[observed]) - 1, -1, -1):
                if not (entities_observed[observed][entity].count("/en/") == 2):
                    del entities_observed[observed][entity]
                else:
                    for perceived_entity in self.perceived:
                        if (('/c/en/' + perceived_entity + '/') in entities_observed[observed][entity]):
                            entities_observed[observed][entity] = entities_observed[observed][entity].replace(
                                '/c/en/' + perceived_entity + '/', '')
                        if (('/c/en' + perceived_entity + '/n/') in entities_observed[observed][entity]):
                            entities_observed[observed][entity] = entities_observed[observed][entity].replace(
                                '/c/en/' + perceived_entity + '/n/', '')

        for observed in entities_observed:
            annotator = 0
            for i in entities_observed[observed]:
                if ('wn/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('wn/', '_')
                if ('/n/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('/n/', '')
                if ('/v/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('/v/', '')
                if ('/a/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('/a/', '')
                if ('/c/en/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('/c/en/',
                                                                                                            '')
                if ('n/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('n/', '')
                if ('/' in i):
                    entities_observed[observed][annotator] = entities_observed[observed][annotator].replace('/', '')
                annotator += 1

        for observed in entities_observed:
            entities_observed[observed] = self.Remove(entities_observed[observed])
            for i in entities_observed[observed]:
                if self.hasNumbers(i) == True:
                    entities_observed[observed].remove(i)

        helper_list_weights = []
        for property in entities_observed:
            for entity in entities_observed[property]:
                helper_list_weights.append(entity)

        list_with_weights = []
        for entity in helper_list_weights:
            for weight in perceived_weights:
                if entity in weight[0]:
                    list_with_weights.append((entity, weight[1]))

        return entities_observed, list_with_weights

    def grounding(self, cleaned_entities, weigths_of_entities):
        porter = PorterStemmer()
        weigths_of_entities = self.Remove(weigths_of_entities)
        helperWeights = []
        for w in weigths_of_entities:
          if w[1] == 1.0 and "_" not in w[0] and str(w[0]) != str(self.perceived[0]):
              helperWeights.append((w[0], w[1]))
        
        helperWeights = self.Remove(helperWeights)
        helperNew = []
        for h in helperWeights:
            syn = wn.synsets(str(h[0]))
            if len(syn) > 1:
                helperNew.append(h)
                
        return helperNew 

In [ ]:
def myfunc(term):
    return 'http://api.conceptnet.io/c/en/' + term + '?offset=0&limit=1000'


def helperCN(c):

    id = myfunc(c)
    respone = requests.get(id)
    objAction = respone.json()
    lst1 = [relation['rel']['@id'] for relation in objAction['edges']]
    lst2 = [relation['@id'] for relation in objAction['edges']]
    lst3 = [relation['weight'] for relation in objAction['edges']]

    return lst1, lst2, lst3

In [ ]:
def conceptNet(word):
  keyword = word.split(" ")
  listWord = []
  for t in keyword:
    cleaned_weights, allData = [], []
    data1, data2, data3 = helperCN(t)
    prp = findSimilarity(data2, data1, [t], data3)
    first = prp.cleaning_entities()
    second, weights = prp.cleaning_entities2(first[1], first[0])
    cleaned_weights = prp.grounding(second, weights)
    listWord.append((t,cleaned_weights[:3]))

  listWord = Remove(listWord)
  return listWord


In [ ]:
#recommendationWN = wordNet('meat beef bike')##given a string, each word must be seperated by a space, it will then return the synonyms and the similarity score from wordnet for each word
#print(recommendationWN)
#recommendationCN = conceptNet('meat beef bike')##given a string, each word must be seperated by a space, it will then return the synonyms and the similarity score from conceptnet for each word, I have restricted it to 1.0 (top recommendations) otherwise there is to much noise
#print(recommendationCN)

In [ ]:
nltk.download('stopwords')

def simpleCN_list2(terms, threshold):
    objects = [requests.get('http://api.conceptnet.io/related/c/en/'+t+'?filter=/c/en').json() for t in terms]
    llist = [[re.sub('/c/en/','',rel['@id']).split('_')for rel in obj['related'] if rel['weight'] > threshold] for obj in objects]

    llist = [elem for sublist in llist for ind_list in sublist for elem in ind_list if elem not in stopwords.words('english') and elem not in terms ]
    return llist

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
